In [1]:
import matplotlib.pyplot as plt
import pycircstat as pycircstat
from matplotlib.ticker import FormatStrFormatter
import numpy as np
import pandas as pd
from openpyxl import load_workbook
from matplotlib import rcParams
rcParams['font.sans-serif']=["arial"]

Here we write this code using V1 tables as an exmaple, but this was also used the same for the hippo and human data

In [2]:
#Opening pickled dictionary
import time
import pickle
start_time = time.time()

pictable_V1Pyr_pd = "../data/pickletable_V1Pyr_pd"
pictable_V1Pvalb_pd = "../data/pickletable_V1Pvalb_pd"
pictable_V1SST_pd = "../data/pickletable_V1SST_pd"

#open the file for reading
open_V1Pyr_obj_pd = open(pictable_V1Pyr_pd, 'rb')
open_V1Pvalb_obj_pd= open(pictable_V1Pvalb_pd, 'rb')
open_V1SST_obj_pd = open(pictable_V1SST_pd, 'rb')

#load the object from the file into new_table_PC
all_PCs = pickle.load(open_V1Pyr_obj_pd)
all_Pvalbs = pickle.load(open_V1Pvalb_obj_pd)
all_SSTs= pickle.load(open_V1SST_obj_pd)

print("Done")
print("This code took", "%.2f"%(time.time() - start_time), "seconds to run.")

Done
('This code took', '19.63', 'seconds to run.')


So for this analysis, the code was run per cell. (In the future some code editing should be able to somehow batch-process this). For now, we start by picking one cell at a time, here is an example of picking one cell in the dataframe:

In [3]:
#all_PCs["filename"].unique()
all_Pvalbs["filename"].unique()
#all_SSTs["filename"].unique()


array(['2021_01_21_121307', '2018_08_16_145013', '2021_01_19_154053',
       '2020_08_13_105226', '2018_10_15_131317', '2018_09_25_101938',
       '2018_09_26_114951', '2019_04_18_142336', '2018_07_25_113957',
       '2018_08_23_134745', '2019_03_06_133645', '2018_10_18_131502',
       '2019_10_04_120017', '2018_08_02_120932', '2018_10_09_104255',
       '2018_08_30_095941', '2018_11_30_131257', '2018_11_06_105219',
       '2021_01_08_180257', '2018_08_14_124959', '2018_08_30_135347',
       '2021_01_22_114535', '2020_10_29_123417', '2021_01_21_151339',
       '2018_08_02_154442'], dtype=object)

### We will look at the control and ES-applied data in different groups (because of the way they are recorded), but the data will be combined into one group when plotting

### This is making a dataframe of suprathreshold data, where ES is applied (i.e. NOT control)

In [4]:
# when there is ES (ES is non-zero)
supra_PCs = all_PCs[(all_PCs['num_spikes_A']>0) & (all_PCs['ex_amp_nA']!=0) & (all_PCs['ex_dur_ms']==13000) & (all_PCs['ex_el_distance_mu']==50)]
supra_Pvalbs = all_Pvalbs[(all_Pvalbs['num_spikes_A']>0) & (all_Pvalbs['ex_amp_nA']!=0) & (all_Pvalbs['ex_dur_ms']==13000) & (all_Pvalbs['ex_el_distance_mu']==50)]
supra_SSTs = all_SSTs[(all_SSTs['num_spikes_A']>0) & (all_SSTs['ex_amp_nA']!=0) & (all_SSTs['ex_dur_ms']==13000) & (all_SSTs['ex_el_distance_mu']==50)]

In [5]:
pd.options.mode.chained_assignment = None #for now, will set this to none, to ignore warning

supra_PCs['spike_ISI'] = supra_PCs.apply( lambda row: [1000./ (x-y) for x,y in zip(row['spike_tt_A'][1:], row['spike_tt_A'])], axis=1)
supra_PCs['spike_ISI'] = supra_PCs.apply(lambda row: row['spike_ISI'], axis=1)
supra_Pvalbs['spike_ISI'] = supra_Pvalbs.apply( lambda row: [1000./ (x-y) for x,y in zip(row['spike_tt_A'][1:], row['spike_tt_A'])], axis=1)
supra_Pvalbs['spike_ISI'] = supra_Pvalbs.apply(lambda row: row['spike_ISI'], axis=1)
supra_SSTs['spike_ISI'] = supra_SSTs.apply( lambda row: [1000./ (x-y) for x,y in zip(row['spike_tt_A'][1:], row['spike_tt_A'])], axis=1)
supra_SSTs['spike_ISI'] = supra_SSTs.apply(lambda row: row['spike_ISI'], axis=1)

In [6]:
onecell_table_supraES = supra_Pvalbs[supra_Pvalbs["filename"]=='2021_01_22_114535']
cell_ID = supra_Pvalbs["filename"].iloc[0]

### This is adding the instaneous spike rate (from ISI) to the dataframe

### Bootstrapped mean VL values:

Here, we are getting the boostrapped mean VL values of each cell by:  
1) taking the vector lengths of spike dataset per cell  
2) binning the spikes according to their instantaneous spike frequency  
3) then getting bootstrap mean values for each cell in each cell class, by shuffling the number of spikes in 10000 trials.  
4) these values are output into an excel spreadsheet, that will then be used for plotting

In [7]:
#makes a condensed dataframe, and you can set the bin sizes with Hz_int_min and Hz_int_max

def return_rose_data(table, ISI=False, n_sample=None,Hz_int_min=None, Hz_int_max=None):
    import random
    data = pd.DataFrame(columns=['ex_frequency', 'ex_amp_nA', 'p_value', 'v_l', 'v_l_std', 'total_n_spikes', 
                                 'mean_firing_rate', 'std_firing_rate', 'v_angle', 'all_spike_phases'])
    
    for (freq,ex_amp),g in table.groupby(['ex_frequency', 'ex_amp_nA']):
        if ISI:
            list1 = g['spike_phase_A_corrected'].sum()
            list2 = g['spike_ISI'].sum()
            ndx= [list2.index(x) for x in list2 if (x>Hz_int_min) & (x<Hz_int_max)]
            all_spike_phases = [list1[x] for x in ndx]
        else:
            all_spike_phases = g['spike_phase_A_corrected'].sum()
        if n_sample: all_spike_phases = random.sample(all_spike_phases, n_sample)
        total_n_spikes = len(all_spike_phases)
        if (len(all_spike_phases) != 0):
            v_angle = pycircstat.descriptive.mean(np.array(all_spike_phases))
            v_angle_std = pycircstat.descriptive.std(np.array(all_spike_phases))
            v_length = pycircstat.descriptive.vector_strength(np.array(all_spike_phases)) 
            p_value = pycircstat.rayleigh(np.array(all_spike_phases))[0]
            data = data.append({'ex_frequency' : freq,
                                'all_spike_phases': all_spike_phases,
                                'ex_amp_nA' : ex_amp, 
                                'p_value': p_value,
                                'v_l': v_length,
                                'v_l_std': pycircstat.descriptive.std(np.array(all_spike_phases)),
                                'total_n_spikes': len(all_spike_phases),
                                'v_angle': v_angle * 180 / np.pi,
                                'v_angle': v_angle_std * 180 / np.pi} ,ignore_index = True)
    return data

In [8]:
#essentially picks out which frequency and amps you want from condensed dataframe
#n_sample is size of population: for Pyr cell, n_sample is 10, for Pvalb cells, it is 50, for SST cells, it is 10
#rangenum is number of trials (in our case, 10,000 trials)

def bootstrap_rose(rose_data, amp_list , fq_list, n_sample=None, rangenum=None):
    global vlmean
    global vangmean
    global p_values_mean
    global confint_vl
    global confint_ang
    global confint_pvals
    global se_vl
    global se_ang
    global se_pval
    global num_of_spikes
    amping = []
    maxnum=[]
    Number_of_spikes=[]
    Vector_Length=[]
    Vector_Angle=[]
    p_values=[]
    t = rose_data[(rose_data['ex_amp_nA']==amp_list[0]) & (rose_data['ex_frequency']==fq_list[0])]
    all_spikes = t['all_spike_phases'].sum()
    if not all_spikes :
#         if len(all_spikes) == 0:
        print 'There were no spikes at amplitude {} and frequency {}.'.format(200, 140)        
    if n_sample:
            for i in range(rangenum):
                if not all_spikes:
                    continue
                reps = np.random.choice(all_spikes, n_sample, replace=True)
                total_n_spikes = len(reps)
                v_angle = pycircstat.descriptive.mean(np.array(reps))
                v_length = pycircstat.descriptive.vector_strength(np.array(reps))
                p_value = pycircstat.tests.rayleigh(np.array(reps))[0]
                Number_of_spikes.append(total_n_spikes)    
                Vector_Length.append(v_length)
                Vector_Angle.append(v_angle * 180./np.pi)
                p_values.append(p_value)
                amping.append(amp_list[0])          
            vlmean = np.mean(Vector_Length)
            vangmean = np.mean(Vector_Angle)
            p_values_mean = np.mean(p_values)
            #CIs
    if len(Vector_Length)>1:
        confint_vl = np.percentile(Vector_Length, [2.5,97.5])            
        confint_ang = np.percentile(Vector_Angle, [2.5,97.5])
        confint_pvals = np.percentile(p_values, [2.5,97.5])
        se_vl = np.std(Vector_Length)
        se_ang = np.std(Vector_Angle)
        se_pval = np.std(p_values)
        num_of_spikes = len(all_spikes)
    elif len(Vector_Length) < 1:
        confint_vl = "no value"            
        confint_ang = "no value"  
        confint_pvals = "no value"  
        se_vl = "no value"  
        se_ang = "no value"  
        se_pval = "no value"  
        num_of_spikes = "no value"
    return vlmean, vangmean, p_values_mean, confint_vl, confint_ang, confint_pvals, se_vl, se_ang, se_pval, num_of_spikes

## This next section outputs the numbers into an excel sheet
Note that: 
1) you need to have the **named excel sheet (WITH the column names included)** already existing, as this code appends to the excel sheet

In [9]:
#nsamp is size of population: for Pyr cell, n_sample is 10, for Pvalb cells, it is 50, for SST cells, it is 10
#cell_ID is defined above, it's the name of the cell you're looking at right now
#The first several lines are defining the boundaries of the frequences we want to look at, and the bin sizes.

def boot_ESapply_output(mytable, mybootstrapfile, Hzbounds, Hzintervals, freq, nsamp):
    for gapped in Hzbounds:
        for interval in Hzintervals:     
            freq = freq
            nsamp = nsamp
            centerfreq = freq + gapped
            Hz_int_minvalue= centerfreq - interval
            Hz_int_maxvalue= centerfreq + interval           

            print "This is for "+ str(Hz_int_minvalue)+ " to "+ str(Hz_int_maxvalue)+" Hz at CenterFreq " + str(centerfreq)
            vlmean, vangmean, p_values_mean, conf_vl, conf_ang, conf_pvals, se_vls, se_angs, se_pvals, num_of_spikes = bootstrap_rose(return_rose_data(mytable, ISI=True, Hz_int_min=Hz_int_minvalue, Hz_int_max=Hz_int_maxvalue), [200] , [freq], n_sample=nsamp, rangenum=10000)

            # make new dataframe with same columns            
            list_of_stuff = {"cell_ID": [cell_ID], "Freq_Hz": [centerfreq], "Amp_nA": 200, "spike_samp_size": nsamp, "Vector_Length": vlmean, "Vector_Angle": vangmean, "p_values": p_values_mean, "Hz_int_minvalue": [Hz_int_minvalue], "Hz_int_maxvalue":[Hz_int_maxvalue], "Conf_VL_low": conf_vl[0], "Conf_VL_ninetyfive": conf_vl[1], "Conf_Angle_low": conf_ang[0], "Conf_Angle_ninetyfive": conf_ang[1], "Conf_pvals_low": conf_pvals[0],  "Conf_pvals_ninetyfive": conf_pvals[1], "se_vls": se_vls, "se_angs": se_angs, "se_pvals": se_pvals, "num_spikes": num_of_spikes}

            # create dataframe 
            mycolumns = ["cell_ID", "Freq_Hz", "Amp_nA", "spike_samp_size", "Vector_Length", "Vector_Angle", "p_values", "Hz_int_minvalue", "Hz_int_maxvalue", "Conf_VL_low", "Conf_VL_ninetyfive",  "Conf_Angle_low", "Conf_Angle_ninetyfive", "Conf_pvals_low", "Conf_pvals_ninetyfive", "se_vls", "se_angs", "se_pvals", "num_spikes"]
            givemenums = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in list_of_stuff.iteritems()]), columns = mycolumns )    
            pd.set_option('precision', 3)
            writer = pd.ExcelWriter(mybootstrapfile, engine='openpyxl')

            #open existing workbook
            writer.book = load_workbook(mybootstrapfile)

            # copy existing sheets
            writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)

            # read existing file
            reader = pd.read_excel(mybootstrapfile)

            # write out the new sheet
            givemenums.to_excel(writer, index=False, header=False, startrow=len(reader)+1, na_rep='NA')
            writer.close()
            givemenums

In [10]:
#For Pyr cells:
mytable = onecell_table_supraES
boot_output_name = "../tables/bootstrap_output_template2.xlsx"
Hzbounds = [-6,-4,-2,0,2,4,6]
Hzintervals = [6,5.5,5,4.5,4,3.5,3,2.5,2,1.5,1,0.5]     
freq = 8
nsamp = 10
        
boot_ESapply_output(mytable, boot_output_name, Hzbounds, Hzintervals, freq, nsamp)

This is for -4 to 8 Hz at CenterFreq 2
There were no spikes at amplitude 200 and frequency 140.
This is for -3.5 to 7.5 Hz at CenterFreq 2
There were no spikes at amplitude 200 and frequency 140.
This is for -3 to 7 Hz at CenterFreq 2
There were no spikes at amplitude 200 and frequency 140.
This is for -2.5 to 6.5 Hz at CenterFreq 2
There were no spikes at amplitude 200 and frequency 140.


C:\Users\Koza\anaconda3\envs\py27\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Koza\anaconda3\envs\py27\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


This is for -2 to 6 Hz at CenterFreq 2
There were no spikes at amplitude 200 and frequency 140.
This is for -1.5 to 5.5 Hz at CenterFreq 2
There were no spikes at amplitude 200 and frequency 140.
This is for -1 to 5 Hz at CenterFreq 2
There were no spikes at amplitude 200 and frequency 140.
This is for -0.5 to 4.5 Hz at CenterFreq 2
There were no spikes at amplitude 200 and frequency 140.
This is for 0 to 4 Hz at CenterFreq 2
There were no spikes at amplitude 200 and frequency 140.
This is for 0.5 to 3.5 Hz at CenterFreq 2
There were no spikes at amplitude 200 and frequency 140.
This is for 1 to 3 Hz at CenterFreq 2
There were no spikes at amplitude 200 and frequency 140.
This is for 1.5 to 2.5 Hz at CenterFreq 2
There were no spikes at amplitude 200 and frequency 140.
This is for -2 to 10 Hz at CenterFreq 4
There were no spikes at amplitude 200 and frequency 140.
This is for -1.5 to 9.5 Hz at CenterFreq 4
There were no spikes at amplitude 200 and frequency 140.
This is for -1 to 9 Hz 

In [11]:
#For Pvalb cells:
mytable = onecell_table_supraES
boot_output_name = "../tables/bootstrap_output_template2.xlsx"
Hzbounds = [-60,-45,-30,-15, 0, 15, 30, 45, 60]
Hzintervals =  [50,20,15,10,5,2]    
freq = 140
nsamp = 50

boot_ESapply_output(mytable, boot_output_name, Hzbounds, Hzintervals, freq, nsamp)

This is for 30 to 130 Hz at CenterFreq 80
This is for 60 to 100 Hz at CenterFreq 80
This is for 65 to 95 Hz at CenterFreq 80
This is for 70 to 90 Hz at CenterFreq 80


C:\Users\Koza\anaconda3\envs\py27\lib\site-packages\pycircstat-0.0.2-py2.7.egg\pycircstat\descriptive.py:487: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(-2 * np.log(r))


This is for 75 to 85 Hz at CenterFreq 80
This is for 78 to 82 Hz at CenterFreq 80
This is for 45 to 145 Hz at CenterFreq 95
This is for 75 to 115 Hz at CenterFreq 95
This is for 80 to 110 Hz at CenterFreq 95
This is for 85 to 105 Hz at CenterFreq 95
This is for 90 to 100 Hz at CenterFreq 95
This is for 93 to 97 Hz at CenterFreq 95
This is for 60 to 160 Hz at CenterFreq 110
This is for 90 to 130 Hz at CenterFreq 110
This is for 95 to 125 Hz at CenterFreq 110
This is for 100 to 120 Hz at CenterFreq 110
This is for 105 to 115 Hz at CenterFreq 110
This is for 108 to 112 Hz at CenterFreq 110
This is for 75 to 175 Hz at CenterFreq 125
This is for 105 to 145 Hz at CenterFreq 125
This is for 110 to 140 Hz at CenterFreq 125
This is for 115 to 135 Hz at CenterFreq 125
This is for 120 to 130 Hz at CenterFreq 125
This is for 123 to 127 Hz at CenterFreq 125
This is for 90 to 190 Hz at CenterFreq 140
This is for 120 to 160 Hz at CenterFreq 140
This is for 125 to 155 Hz at CenterFreq 140
This is for 

In [12]:
#For SST cells:
mytable = onecell_table_supraES
boot_output_name = "../tables/bootstrap_output_template2.xlsx"
Hzbounds = [-20,-15,-10,-5,0,5,10,15,20]
Hzintervals = [15, 13, 11, 9, 7, 5, 3, 1]     
freq = 30
nsamp = 10

boot_ESapply_output(mytable, boot_output_name, Hzbounds, Hzintervals, freq, nsamp)

This is for -5 to 25 Hz at CenterFreq 10
There were no spikes at amplitude 200 and frequency 140.
This is for -3 to 23 Hz at CenterFreq 10
There were no spikes at amplitude 200 and frequency 140.
This is for -1 to 21 Hz at CenterFreq 10
There were no spikes at amplitude 200 and frequency 140.
This is for 1 to 19 Hz at CenterFreq 10
There were no spikes at amplitude 200 and frequency 140.
This is for 3 to 17 Hz at CenterFreq 10
There were no spikes at amplitude 200 and frequency 140.
This is for 5 to 15 Hz at CenterFreq 10
There were no spikes at amplitude 200 and frequency 140.
This is for 7 to 13 Hz at CenterFreq 10
There were no spikes at amplitude 200 and frequency 140.
This is for 9 to 11 Hz at CenterFreq 10
There were no spikes at amplitude 200 and frequency 140.
This is for 0 to 30 Hz at CenterFreq 15
There were no spikes at amplitude 200 and frequency 140.
This is for 2 to 28 Hz at CenterFreq 15
There were no spikes at amplitude 200 and frequency 140.
This is for 4 to 26 Hz at C

### This is making a dataframe of suprathreshold data, but ONLY THE CONTROL (i.e. ES amplitude is 0 nA)

In the experimental protocol, there is one 0 nA sweep for each amplitude (not each frequency).
i.e. in a set with 200 nA stim, there is one 0 nA sweep, then sweeps at different frequencies.
So for comparison at the different frequencies, control_supra_PCs takes that first 0 nA sweep and lists it 4 times for every frequency. 

Our goal here is to find the sweep that is the control sweep associated with the 200 nA ES protocol.      
1) Pick a frequency and then look at the sweeps.  
2) we need to find which control sweep (sweep_number) is associated with the 200 nA ES (i.e. the one that came before it)  
3) then make a table contains just that 200 nA-control sweep, and add the spike_ISI info

*Note, the labeling is a bit confusing--a control sweep might has a "frequency" labeled with it, but it's just for labeling to say that that control sweep is being compared to an ES-sweep that has that ES-frequency

In [13]:
onecell_table_allsupra = all_Pvalbs[all_Pvalbs["filename"]=='2021_01_22_114535']
cell_ID = onecell_table_allsupra["filename"].iloc[0]

freq_to_check = 140
freq_to_check_table = onecell_table_allsupra[(onecell_table_allsupra["ex_frequency"]==freq_to_check) & (onecell_table_allsupra['num_spikes_A']>0) & (onecell_table_allsupra['ex_dur_ms']==13000) & (onecell_table_allsupra['ex_el_distance_mu']==50)]
freq_to_check_table

,filename,sweep_id,sweep_number,ex_el_id,stim_el_id,in_el_id,ex_el_distance_mu,spike_tt,ex_amp_nA,ex_frequency,...,vi_phase,vm_amp_mV,vm_phase,avg_vm_mV,spike_phase,spike_tt_A,spike_phase_A,num_spikes,num_spikes_A,spike_phase_A_corrected
4565,2021_01_22_114535,98_94_7_0_141_260,94.0,7.0,2.0,5.0,50.0,"[3006.26, 3012.82, 3019.9800000000005, 3028.00...",0.0,140.0,...,NaN,NaN,NaN,NaN,"[0.7296432253790561, 0.22971491888842577, 0.30...","[3505.78, 3513.88, 3521.5600000000004, 3529.26...","[-2.872073793796031, -1.9611476230657197, -1.4...",1138,1126,"[1.840315186588659, 2.7512413573189702, 3.2453..."
4569,2021_01_22_114535,98_7_200_141_260,98.0,7.0,2.0,5.0,50.0,"[3005.58, 3012.02, 3018.76, 3025.5400000000004...",200.0,140.0,...,NaN,NaN,NaN,NaN,"[0.12305151981614593, -0.47551386416075303, -0...","[3502.3800000000006, 3514.8400000000006, 3523....","[0.4218275892322678, -1.1054122279670295, -0.0...",1250,1186,"[5.1342165696169575, 3.60697675241766, 4.64195..."
4570,2021_01_22_114535,103_99_7_0_141_260,99.0,7.0,2.0,5.0,50.0,"[3006.9600000000005, 3018.6800000000003, 3028....",0.0,140.0,...,NaN,NaN,NaN,NaN,"[1.3820896140178585, -0.8377965735922697, 1.38...","[3501.6000000000004, 3508.96, 3516.86, 3523.84...","[-0.28259793351503815, -0.04189275194770081, 0...",1213,1194,"[4.429791046869651, 4.670496228436988, 5.36430..."
4577,2021_01_22_114535,103_7_100_141_260,103.0,7.0,2.0,5.0,50.0,"[3005.5400000000004, 3012.82, 3019.84, 3026.84...",100.0,140.0,...,NaN,NaN,NaN,NaN,"[0.13254780443523195, 0.2647795454769408, 0.21...","[3504.8, 3512.0600000000004, 3519.520000000000...","[2.597038804602266, 2.7372393138451567, 3.0190...",1269,1259,"[1.0262424778073698, 1.16644298705026, 1.44826..."
4579,2021_01_22_114535,109_105_7_0_141_260,105.0,7.0,2.0,5.0,50.0,"[3004.9, 3011.5800000000004, 3018.200000000000...",0.0,140.0,...,NaN,NaN,NaN,NaN,"[-0.5628560933398248, -0.8118627428594807, -1....","[3502.7200000000007, 3509.4200000000005, 3516....","[0.7018079717654437, 0.3073021644768468, 0.631...",1337,1271,"[5.414196952150133, 5.019691144861536, 5.34371..."
4585,2021_01_22_114535,109_7_50_141_260,109.0,7.0,2.0,5.0,50.0,"[3004.48, 3010.7800000000007, 3017.32, 3024.34...",50.0,140.0,...,NaN,NaN,NaN,NaN,"[-0.9338925121659738, -1.5775747470665231, -2....","[3506.4, 3513.2000000000003, 3520.400000000000...","[-2.217628949341022, -2.511469773421509, -2.42...",1367,1296,"[2.4947600310436675, 2.2009192069631807, 2.290..."
4589,2021_01_22_114535,114_110_7_0_141_260,110.0,7.0,2.0,5.0,50.0,"[3005.26, 3012.2400000000002, 3020.08, 3027.14...",0.0,140.0,...,NaN,NaN,NaN,NaN,"[0.10552197268693186, -0.12365936859441629, 0....","[3859.5400000000004, 3867.1600000000003, 3874....","[2.8901757064394236, -3.132236057612157, -2.62...",1243,1216,"[1.3193793796445266, 1.5801529227725326, 2.086..."
4593,2021_01_22_114535,114_7_25_141_260,114.0,7.0,2.0,5.0,50.0,"[3004.82, 6376.84, 6384.9800000000005, 6392.68...",25.0,140.0,...,NaN,NaN,NaN,NaN,"[-0.3923420291664099, 2.318385784647284, 3.061...","[6376.84, 6384.9800000000005, 6392.68, 6400.6,...","[2.318385784647284, 3.0614978713413334, -2.535...",628,627,"[0.7475894578523875, 1.4907015445464373, 2.177..."


In [14]:
sweep_number_200_con = 94

only200_con_freq_table= freq_to_check_table[(freq_to_check_table["sweep_number"]==sweep_number_200_con)]
only200_con_freq_table['spike_ISI'] = only200_con_freq_table.apply( lambda row: [1000./ (x-y) for x,y in zip(row['spike_tt_A'][1:], row['spike_tt_A'])], axis=1)
only200_con_freq_table['spike_ISI'] = only200_con_freq_table.apply(lambda row: row['spike_ISI'], axis=1)

only200_con_freq_table

,filename,sweep_id,sweep_number,ex_el_id,stim_el_id,in_el_id,ex_el_distance_mu,spike_tt,ex_amp_nA,ex_frequency,...,vm_amp_mV,vm_phase,avg_vm_mV,spike_phase,spike_tt_A,spike_phase_A,num_spikes,num_spikes_A,spike_phase_A_corrected,spike_ISI
4565,2021_01_22_114535,98_94_7_0_141_260,94.0,7.0,2.0,5.0,50.0,"[3006.26, 3012.82, 3019.9800000000005, 3028.00...",0.0,140.0,...,NaN,NaN,NaN,"[0.7296432253790561, 0.22971491888842577, 0.30...","[3505.78, 3513.88, 3521.5600000000004, 3529.26...","[-2.872073793796031, -1.9611476230657197, -1.4...",1138,1126,"[1.840315186588659, 2.7512413573189702, 3.2453...","[123.45679012345818, 130.2083333333284, 129.87..."


In [15]:
def boot_CON_output(mytable, mybootstrapfile, Hzbounds, Hzintervals, freq, nsamp):
    for gapped in Hzbounds:
        for interval in Hzintervals:     
            freq = freq
            nsamp = nsamp
            centerfreq = freq + gapped
            Hz_int_minvalue= centerfreq - interval
            Hz_int_maxvalue= centerfreq + interval           

            print "This is for "+ str(Hz_int_minvalue)+ " to "+ str(Hz_int_maxvalue)+" Hz at CenterFreq " + str(centerfreq)
            vlmean, vangmean, p_values_mean, conf_vl, conf_ang, conf_pvals, se_vls, se_angs, se_pvals, num_of_spikes = bootstrap_rose(return_rose_data(mytable, ISI=True, Hz_int_min=Hz_int_minvalue, Hz_int_max=Hz_int_maxvalue), [0] , [freq], n_sample=nsamp, rangenum=10000)

            # make new dataframe with same columns
            
            list_of_stuff = {"cell_ID": [cell_ID], "Freq_Hz": [centerfreq], "Amp_nA": 0, "spike_samp_size": nsamp, "Vector_Length": vlmean, "Vector_Angle": vangmean, "p_values": p_values_mean, "Hz_int_minvalue": [Hz_int_minvalue], "Hz_int_maxvalue":[Hz_int_maxvalue], "Conf_VL_low": conf_vl[0], "Conf_VL_ninetyfive": conf_vl[1], "Conf_Angle_low": conf_ang[0], "Conf_Angle_ninetyfive": conf_ang[1], "Conf_pvals_low": conf_pvals[0],  "Conf_pvals_ninetyfive": conf_pvals[1], "se_vls": se_vls, "se_angs": se_angs, "se_pvals": se_pvals, "num_spikes": num_of_spikes}

            # create dataframe 
            mycolumns = ["cell_ID", "Freq_Hz", "Amp_nA", "spike_samp_size", "Vector_Length", "Vector_Angle", "p_values", "Hz_int_minvalue", "Hz_int_maxvalue", "Conf_VL_low", "Conf_VL_ninetyfive",  "Conf_Angle_low", "Conf_Angle_ninetyfive", "Conf_pvals_low", "Conf_pvals_ninetyfive", "se_vls", "se_angs", "se_pvals", "num_spikes"]
            givemenums = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in list_of_stuff.iteritems()]), columns = mycolumns )    
            pd.set_option('precision', 3)
            writer = pd.ExcelWriter(mybootstrapfile, engine='openpyxl')

            #open existing workbook
            writer.book = load_workbook(mybootstrapfile)

            # copy existing sheets
            writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)

            # read existing file
            reader = pd.read_excel(mybootstrapfile)

            # write out the new sheet
            givemenums.to_excel(writer, index=False, header=False, startrow=len(reader)+1, na_rep='NA')
            writer.close()
            givemenums

In [16]:
#For Pyr cells:
mytable = onecell_table_supraES
boot_output_name = "../tables/bootstrap_output_template2.xlsx"
Hzbounds = [-6,-4,-2,0,2,4,6]
Hzintervals = [6,5.5,5,4.5,4,3.5,3,2.5,2,1.5,1,0.5]  
freq = 8
nsamp = 10

boot_CON_output(mytable, boot_output_name, Hzbounds, Hzintervals, freq, nsamp)

This is for -4 to 8 Hz at CenterFreq 2
There were no spikes at amplitude 200 and frequency 140.
This is for -3.5 to 7.5 Hz at CenterFreq 2
There were no spikes at amplitude 200 and frequency 140.
This is for -3 to 7 Hz at CenterFreq 2
There were no spikes at amplitude 200 and frequency 140.
This is for -2.5 to 6.5 Hz at CenterFreq 2
There were no spikes at amplitude 200 and frequency 140.
This is for -2 to 6 Hz at CenterFreq 2
There were no spikes at amplitude 200 and frequency 140.
This is for -1.5 to 5.5 Hz at CenterFreq 2
There were no spikes at amplitude 200 and frequency 140.
This is for -1 to 5 Hz at CenterFreq 2
There were no spikes at amplitude 200 and frequency 140.
This is for -0.5 to 4.5 Hz at CenterFreq 2
There were no spikes at amplitude 200 and frequency 140.
This is for 0 to 4 Hz at CenterFreq 2
There were no spikes at amplitude 200 and frequency 140.
This is for 0.5 to 3.5 Hz at CenterFreq 2
There were no spikes at amplitude 200 and frequency 140.
This is for 1 to 3 Hz 

In [17]:
#For Pvalb cells
mytable = only200_con_freq_table
boot_output_name = "../tables/bootstrap_output_template2.xlsx"
Hzbounds = [-60,-45,-30,-15, 0, 15, 30, 45, 60]
Hzintervals = [50,20,15,10,5,2]
freq = 140
nsamp = 50

boot_CON_output(mytable, boot_output_name, Hzbounds, Hzintervals, freq, nsamp)

This is for 30 to 130 Hz at CenterFreq 80
This is for 60 to 100 Hz at CenterFreq 80
This is for 65 to 95 Hz at CenterFreq 80
This is for 70 to 90 Hz at CenterFreq 80
This is for 75 to 85 Hz at CenterFreq 80
This is for 78 to 82 Hz at CenterFreq 80
This is for 45 to 145 Hz at CenterFreq 95
This is for 75 to 115 Hz at CenterFreq 95
This is for 80 to 110 Hz at CenterFreq 95
This is for 85 to 105 Hz at CenterFreq 95
This is for 90 to 100 Hz at CenterFreq 95
This is for 93 to 97 Hz at CenterFreq 95
This is for 60 to 160 Hz at CenterFreq 110
This is for 90 to 130 Hz at CenterFreq 110
This is for 95 to 125 Hz at CenterFreq 110
This is for 100 to 120 Hz at CenterFreq 110
This is for 105 to 115 Hz at CenterFreq 110
This is for 108 to 112 Hz at CenterFreq 110
This is for 75 to 175 Hz at CenterFreq 125
This is for 105 to 145 Hz at CenterFreq 125
This is for 110 to 140 Hz at CenterFreq 125
This is for 115 to 135 Hz at CenterFreq 125
This is for 120 to 130 Hz at CenterFreq 125
This is for 123 to 12

In [18]:
#For SST cells:
mytable = onecell_table_supraES
boot_output_name = "../tables/bootstrap_output_template2.xlsx"
Hzbounds = [-20,-15,-10,-5,0,5,10,15,20]
Hzintervals = [15, 13, 11, 9, 7, 5, 3, 1]    
freq = 30
nsamp = 10

boot_CON_output(mytable, boot_output_name, Hzbounds, Hzintervals, freq, nsamp)

This is for -5 to 25 Hz at CenterFreq 10
There were no spikes at amplitude 200 and frequency 140.
This is for -3 to 23 Hz at CenterFreq 10
There were no spikes at amplitude 200 and frequency 140.
This is for -1 to 21 Hz at CenterFreq 10
There were no spikes at amplitude 200 and frequency 140.
This is for 1 to 19 Hz at CenterFreq 10
There were no spikes at amplitude 200 and frequency 140.
This is for 3 to 17 Hz at CenterFreq 10
There were no spikes at amplitude 200 and frequency 140.
This is for 5 to 15 Hz at CenterFreq 10
There were no spikes at amplitude 200 and frequency 140.
This is for 7 to 13 Hz at CenterFreq 10
There were no spikes at amplitude 200 and frequency 140.
This is for 9 to 11 Hz at CenterFreq 10
There were no spikes at amplitude 200 and frequency 140.
This is for 0 to 30 Hz at CenterFreq 15
There were no spikes at amplitude 200 and frequency 140.
This is for 2 to 28 Hz at CenterFreq 15
There were no spikes at amplitude 200 and frequency 140.
This is for 4 to 26 Hz at C